In [1]:
import youtokentome as yttm

import pandas as pd
import numpy as np

from src.utils import *
from src.vars import *
from src.model import Classifier

from tqdm.notebook import tqdm

import torch

In [2]:
data = pd.read_csv(test_file, sep='\t')

In [3]:
tokenizer = yttm.BPE(model=tokenizer_path)
sep_token = tokenizer.id_to_subword(sep_token_id)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
clued = clue(data, sep_token=sep_token, to_lower=True)

In [5]:
model = torch.load(model_file).to(device)

In [6]:
rows = clued.values
rows = [torch.LongTensor(tokenizer.encode(r)) for r in rows]

In [7]:
def create_dataloader(data, batch_size, pad_id,
                      pin_memory=True, shuffle=True, num_of_workers=0):
    """Create DataLoader object for given data.
    """
    partial_collate = partial(my_collate, pad_id=pad_id)
    return DataLoader(data,
                      batch_size=batch_size,
                      collate_fn=partial_collate,
                      pin_memory=pin_memory,
                      drop_last=False,
                      shuffle=shuffle,
                      num_workers=num_of_workers)

dl = create_dataloader(list(zip(rows, range(len(rows)))), 128, pad_token_id, shuffle=False)

In [8]:
res = []
for v, _ in tqdm(dl):
    v = v.to(device)
    y_hat = model(v)
    res.append(torch.max(y_hat, 1)[1].cpu().detach().numpy())

In [10]:
united = np.concatenate(res)
assert len(united) == len(data)

In [11]:
pd.DataFrame(united, columns=[label_column], dtype=bool).to_csv('result.tsv', index=False, sep='\t')